# Installing Z3 and Imports

In [1]:
'''
!pip install z3-solver
!pip install pandas
!pip install numpy
!pip install sklearn
'''

'\n!pip install z3-solver\n!pip install pandas\n!pip install numpy\n!pip install sklearn\n'

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn import metrics
from z3 import *

# Training SVM

## Data Preprocessing.

In [3]:
cancer = datasets.load_breast_cancer()

In [4]:
df = pd.DataFrame(cancer.data, columns = cancer.feature_names)

In [5]:
normalized_df=(cancer.data-cancer.data.min())/(cancer.data.max()-cancer.data.min())
#normalized_df=(cancer.data-cancer.data.mean())/cancer.data.std()

In [6]:
normalized_df.min(),normalized_df.max()

(0.0, 1.0)

## Data Separation and Training

In [7]:
X_train, X_test, y_train, y_test = train_test_split(normalized_df, cancer.target, test_size=0.3,random_state=107) # 70% training and 30% test

In [8]:
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9064327485380117


In [9]:
y_pred_train = clf.predict(X_train)
print("Accuracy on Training:", metrics.accuracy_score(y_train, y_pred_train))

Accuracy on Training: 0.8592964824120602


## SVM Decision Function For The First Element of Training Dataset

In [10]:
#Sum (coef @ sup_vec @ X[index] + bias)
((clf.dual_coef_ @ clf.support_vectors_) @ X_train[1].reshape(1, len(X_train[1])).T + clf.intercept_)[0][0]

0.7999758604849845

In [11]:
print("Coef / Support Vectors / X_Train.T Reshaped / Intercept (bias)")
clf.dual_coef_.shape, clf.support_vectors_.shape, X_train[1].reshape(1, len(X_train[1])).T.shape, clf.intercept_.shape

Coef / Support Vectors / X_Train.T Reshaped / Intercept (bias)


((1, 194), (194, 30), (30, 1), (1,))

# Defining Thresholds

## Finding Decision Function's Superior and Inferior Limits

In [12]:
decfun = clf.decision_function(X_train)
lim_pos = decfun[np.argmax(decfun)]
lim_neg = decfun[np.argmin(decfun)]
print("Limite Pos = ",lim_pos, '\nLimite Neg =',lim_neg)

Limite Pos =  1.8675370149765804 
Limite Neg = -6.600371124175771


## Defining WR and Rejection Thresholds

In [13]:
wr = []
for i in range(1,13):
  wr.append(i*0.04)
print(wr)

[0.04, 0.08, 0.12, 0.16, 0.2, 0.24, 0.28, 0.32, 0.36, 0.4, 0.44, 0.48]


In [14]:
#Total quantity of elements
n_elements = decfun.shape[0]
n_elements

398

In [15]:
t1 = []
t2 = []
for i in range (1,101):
  t1.append(0.01*i*lim_pos)
  t2.append(0.01*i*lim_neg)

In [16]:
dict_reject = {'WR': wr,'T1':t1, 'T2':t2,'E':[],'R':[],'EWRR': []}

## All Iterations

In [17]:
for i,wr_ in enumerate(dict_reject['WR']):
  EwrR = []
  for i,t in enumerate(dict_reject['T1']):
    #Get Number of Rejected
    positive_indexes = np.where(decfun >= dict_reject['T1'][i])
    negative_indexes = np.where(decfun < dict_reject['T2'][i])
    rejected_indexes = np.where((decfun < dict_reject['T1'][i]) & (decfun >= dict_reject['T2'][i]))
    R = rejected_indexes[0].shape[0]
    #np.array(positive_indexes).shape,np.array(negative_indexes).shape, R

    #Get Number of Misclassifications
    class_p = y_train[positive_indexes]
    class_n = y_train[negative_indexes]
    error_p = np.where(class_p == np.argmin(np.unique(y_train)))[0].shape[0]
    error_n = np.where(class_n == np.argmax(np.unique(y_train)))[0].shape[0]
    error_total = (error_p + error_n)/(n_elements - R)
    
    EwrR.append(error_total + wr_ * R)
    #print('(',i+1,')',"Erro = ",error_total," wr = ",wr_,' R = ',R, 'E + wrR = ',error_total + wr_ * R,' T1 = ',dict_reject['T1'][i], ' T2 = ', dict_reject['T2'][i], ' Error P = ', error_p, ' Error N = ', error_n)
  #print(np.argmin(np.array(EwrR)), EwrR[np.argmin(np.array(EwrR))])

# Implementing SVM function for Z3 Solver

## Z3 Decision Function

In [18]:
np.RealVal = np.vectorize(RealVal) 
np.RealVector = np.vectorize(RealVector) 

In [19]:
z3_dual_coef = np.RealVal(clf.dual_coef_)
z3_support_vectors = np.RealVal(clf.support_vectors_)
z3_intercept_ = np.RealVal(clf.intercept_)
z3_X_Train = np.RealVector('x',X_train.shape[1])

In [20]:
z3_X_Train

array([x__0, x__1, x__2, x__3, x__4, x__5, x__6, x__7, x__8, x__9, x__10,
       x__11, x__12, x__13, x__14, x__15, x__16, x__17, x__18, x__19,
       x__20, x__21, x__22, x__23, x__24, x__25, x__26, x__27, x__28,
       x__29], dtype=object)

# Z3 with Reject Option

In [21]:
#Defining z3 thresholds
T1,T2 = t1[0],t2[0]
rejected_indexes2 = np.where((decfun < T1) & (decfun >= T2))[0]

### Fixed List

In [22]:
def fixed_list(t1, t2, X, z3_coef, z3_sup_vec, z3_X, z3_intercept, reject_indexes, show_values=False):
    satisfatible = []
    unsatisfatible = []
    print("Number: ",len(reject_indexes))
    for i in range(0, len(reject_indexes)):
      #Add Assertions for 0<= feature <= 1
      index_list = list(range(len(z3_X)))
      unsat_list = []
      sat_list = []

      #Select a feature and unfix it
      for z in range(0, len(z3_X)):
        solver2 = Solver()
        solver2.add(Or(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t1,
                ((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t2))
        for j in range(0, len(z3_X)):
          solver2.add(z3_X[j] >=0)
          solver2.add(z3_X[j] <=1)
        for j in range(0, len(z3_X)):
          if j != z: #Choose one to check influence
            solver2.add(z3_X[j] == X[reject_indexes[i]][j])

        check = solver2.check()
        if check == sat:
          sat_list.append(z)
          model = solver2.model()
          if show_values:
              print('i = ',i,z, check, X[reject_indexes[i]][z], model[z3_X[z]].numerator_as_long()/model[z3_X[z]].denominator_as_long())
              #print(z, check)
        else:
          unsat_list.append(z)
        
      print("Finished ",i)
      satisfatible.append(sat_list)
      unsatisfatible.append(unsat_list)
          #index_list.remove(z)
          #print('i = ',i, z, check)
      #print("Relevant: ",sat_list, '\nUnsat List: ',unsat_list,'\n')
    for i in range(0, len(satisfatible)):
        print(i,'\nSatisfatível: ', satisfatible[i],'\nInsatisfatível: ',unsatisfatible[i],'\n\n')
    return satisfatible, unsatisfatible
    

In [23]:
#Limiar Superior, Limiar Inferior, Conjunto de Entradas, Coeficiente, Vetores de Suporte, Entrada Z3, Bias, Lista de Rejeição
#satisfatible, unsatisfatible = fixed_list(T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,rejected_indexes2,True)

### Unfixed List, Crescent

In [24]:
def unfixed_list(t1, t2, X, z3_coef, z3_sup_vec, z3_X, z3_intercept, reject_indexes,crescent_order=True,show_values=False):
    satisfatible = []
    unsatisfatible = []
    global_values = []
    print("Number of Rejected: ",len(reject_indexes))
    if crescent_order:
        for i in range(0, len(reject_indexes)):
          #Add Assertions for 0<= feature <= 1
          index_list = list(range(len(z3_X)))
          unsat_list = []
          sat_list = []
          values = []  

          #Select a feature and unfix it
          for z in range(0, len(z3_X)):
            solver2 = Solver()
            solver2.add(Or(((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] >= t1,
                    ((z3_coef @ z3_sup_vec) @ z3_X.reshape(1, len(z3_X)).T + z3_intercept)[0][0] < t2))
            for j in range(0, len(z3_X)):
              solver2.add(z3_X[j] >=0)
              solver2.add(z3_X[j] <=1)
            used_list = []
            for j in range(0, len(z3_X)):
              if j != z and j in index_list: #Choose one to check influence
                solver2.add(z3_X[j] == X[reject_indexes[i]][j])
                used_list.append(j)

            check = solver2.check()
            if check == sat:         
              model = solver2.model()
              value = model[z3_X[z]].numerator_as_long()/model[z3_X[z]].denominator_as_long()
              sat_list.append(z)
              values.append(value)
              if show_values:  
                  print('i = ',i,z, check, X[reject_indexes[i]][z], value)
            else:
              unsat_list.append(z)
              index_list.remove(z)
              if show_values:
                  print('i = ',i, z, check)  
          print("Finished ",i)
          satisfatible.append(sat_list)
          unsatisfatible.append(unsat_list)
          global_values.append(values)
              
          #print("Relevant: ",sat_list, '\nUnsat List: ',unsat_list,'\n')     
    for i in range(0, len(satisfatible)):
        print('Rejected ',i,'\nSatisfatível: ', satisfatible[i],'\nValores: ',global_values,'\nInsatisfatível: ',unsatisfatible[i],'\n\n')
    return satisfatible, unsatisfatible

In [25]:
#satisfatible, unsatisfatible = unfixed_list(T1,T2,X_train,z3_dual_coef,z3_support_vectors,z3_X_Train,z3_intercept_,rejected_indexes2,crescent_order=True, show_values=True)

# Anchors

## Setting Up